In [ ]:
# %% [markdown]
# # FTEC5660 Homework 02: CV Verification Agent (FIXED)
# This notebook implements an autonomous AI agent for checking CV consistency against a simulated Social Graph.

# %% [markdown]
# ## 1. Setup and Imports

# %%
import os
import json
import time
import base64
from typing import List, Dict, Any, Optional

# Install necessary packages if missing
# !pip install openai

from openai import OpenAI

# ==============================================================================
# CONFIGURATION
# ==============================================================================

CONFIG = {
    # Credentials
    "API_KEY": "sk-TMy2dCghqqQpLU2YTtxLhySEt8Cfybf1iIP7lajvcIp5Hx0k", 
    "BASE_URL": "https://api.yyds168.net/v1",
    
    # Model
    "MODEL_NAME": "gemini-3-pro-preview",
    
    # Resilience
    "MAX_RETRIES": 3,
    "TIMEOUT": 120,
}

print("[SYSTEM] Configuration loaded.")

# %% [markdown]
# ## 2. LLM Client Wrapper (Robust)

# %%
class LLMClient:
    def __init__(self, config):
        self.client = OpenAI(
            api_key=config["API_KEY"],
            base_url=config["BASE_URL"]
        )
        self.model = config["MODEL_NAME"]

    def chat_completion(self, messages: List[Dict], tools: Optional[List[Dict]] = None) -> Any:
        try:
            params = {
                "model": self.model,
                "messages": messages,
                "temperature": 0.0, # Deterministic
            }
            if tools:
                params["tools"] = tools
                params["tool_choice"] = "auto"

            response = self.client.chat.completions.create(**params)
            return response.choices[0].message
        except Exception as e:
            print(f"[LLM ERROR] {str(e)}")
            return None

llm_client = LLMClient(CONFIG)

# %% [markdown]
# ## 3. Mock Social Graph MCP Server
# Simulates the database of "True" profiles.

# %%
class MockSocialGraphServer:
    def __init__(self):
        # Database of "Real" profiles (Ground Truth)
        # Note: These are designed to conflict slightly with the CVs to test the agent.
        
        self.fb_users = [
            {"id": 101, "display_name": "John Smith", "city": "Singapore", "country": "Singapore", "bio": "Just chilling", "friends": [102]},
            {"id": 102, "display_name": "Minh P.", "city": "Hong Kong", "country": "China", "bio": "Designer life", "friends": [101]},
        ]
        
        self.linkedin_profiles = [
            {
                "id": 201, 
                "name": "John Smith", 
                "headline": "Marketing Associate", 
                "location": "Singapore", 
                "experience": [
                    {"company": "ByteDance", "title": "Intern", "start_year": 2020, "end_year": 2021}
                ],
                "education": [
                    {"school": "McGill University", "degree": "BSc Marketing", "start_year": 2005, "end_year": 2009}
                ]
            },
            {
                "id": 202, 
                "name": "Minh Pham", 
                "headline": "Design Lead", 
                "location": "Beijing, China",
                "experience": [
                    {"company": "BCG", "title": "Manager", "start_year": 2022, "is_current": True},
                    {"company": "Tencent", "title": "Analyst", "start_year": 2013, "end_year": 2017}
                ],
                "education": [
                    {"school": "University of Hong Kong", "degree": "BSc Design", "start_year": 2011}
                ]
            },
            {
                "id": 203, 
                "name": "Wei Zhang", 
                "headline": "Consultant", 
                "location": "Munich, Germany",
                "education": [
                    {"school": "University of Tokyo", "degree": "BSc Economics", "start_year": 2015} # Discrepancy: Economics vs Consulting
                ]
            },
            {
                "id": 204, 
                "name": "Rahul Sharma", 
                "headline": "Software Engineer", 
                "location": "Singapore",
                "education": [
                    {"school": "Tsinghua University", "degree": "PhD Computer Science", "start_year": 2021} # Discrepancy: CS vs Legal
                ],
                "experience": [
                    {"company": "Microsoft", "title": "Senior Engineer", "start_year": 2021, "is_current": True}
                ]
            }
        ]

    # --- FACEBOOK TOOLS ---
    def search_facebook_users(self, q: str, limit: int = 20, fuzzy: bool = True):
        results = []
        q_lower = q.lower()
        for user in self.fb_users:
            if q_lower in user["display_name"].lower():
                results.append({
                    "id": user["id"], "display_name": user["display_name"],
                    "city": user["city"], "country": user["country"], "match_type": "exact"
                })
        return json.dumps(results)

    def get_facebook_profile(self, user_id: int):
        for user in self.fb_users:
            if user["id"] == int(user_id):
                return json.dumps(user)
        return json.dumps({"error": "User not found"})

    # --- LINKEDIN TOOLS ---
    def search_linkedin_people(self, q: str, location: str = None, industry: str = None, limit: int = 20, fuzzy: bool = True):
        results = []
        q_lower = q.lower()
        for p in self.linkedin_profiles:
            # Simple keyword matching
            if q_lower in p["name"].lower():
                results.append({
                    "id": p["id"], "name": p["name"], "headline": p["headline"],
                    "location": p["location"], "match_type": "exact"
                })
        return json.dumps(results)

    def get_linkedin_profile(self, person_id: int):
        for p in self.linkedin_profiles:
            if p["id"] == int(person_id):
                return json.dumps(p)
        return json.dumps({"error": "Profile not found"})

mcp_server = MockSocialGraphServer()

# %% [markdown]
# ## 4. Tool Definitions

# %%
tools_schema = [
    {
        "type": "function",
        "function": {
            "name": "search_facebook_users",
            "description": "Find Facebook users by name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "q": {"type": "string", "description": "Name to search"},
                    "limit": {"type": "integer"},
                    "fuzzy": {"type": "boolean"}
                },
                "required": ["q"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_facebook_profile",
            "description": "Get detailed Facebook profile.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {"type": "integer", "description": "ID from search"}
                },
                "required": ["user_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_linkedin_people",
            "description": "Search LinkedIn professionals.",
            "parameters": {
                "type": "object",
                "properties": {
                    "q": {"type": "string", "description": "Name"},
                    "location": {"type": "string"},
                    "industry": {"type": "string"}
                },
                "required": ["q"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_linkedin_profile",
            "description": "Retrieve complete professional profile from LinkedIn.",
            "parameters": {
                "type": "object",
                "properties": {
                    "person_id": {"type": "integer", "description": "ID from search"}
                },
                "required": ["person_id"]
            }
        }
    }
]

# %% [markdown]
# ## 5. Input Data (Exact content from prompt)

# %%
CV_DATA = {
    "CV_1.pdf": """
    Experience
    John Smith
    Marketing Professional
    Singapore, Singapore Kowloon
    Engineer, ByteDance
    • Worked in a fast-paced, global technology environment.
    Education
    McGill University
    Bachelor of Science (BSc) in Marketing
    2020 Present
    Graduated 2009
    """,
    "CV_2.pdf": """
    Minh Pham
    Design Professional
    Beijing, China Hong Kong
    Manager, BCG
    Analyst, Tencent
    Education
    BSc in Design
    The University of Hong Kong
    2022 Present
    2013-2017
    """,
    "CV_3.pdf": """
    Wei Zhang
    Consulting Professional
    2013 Present
    Engineer, PwC
    Education
    Munich, Germany
    Sydney (Hometown)
    BSc in Consulting
    University of Tokyo
    """,
    "CV_4.pdf": """
    Rahul Sharma
    Legal Professional
    Singapore (Hometown) Singapore / Philippines
    Senior Engineer, Microsoft (2021-2027)
    Consultant, StartupXYZ (2020-2023)
    Education
    PhD in Legal Studies
    Tsinghua University
    Skills: Compliance, Litigation, Quantum Computing
    """
}

# %% [markdown]
# ## 6. The Agent Logic (Fixed Serialization)

# %%
class CVVerificationAgent:
    def __init__(self, client, tools):
        self.client = client
        self.tools = tools
        self.system_prompt = """
        You are an expert KYC and Background Check Agent.
        Your goal is to verify a candidate's CV against public social media data (LinkedIn, Facebook).
        
        PROCESS:
        1. Analyze the CV content.
        2. Search for the candidate on LinkedIn (primary) and Facebook.
        3. Retrieve profiles to compare:
           - Job titles/Companies
           - Education (Degree, School)
           - Location
        4. Identify discrepancies between CV and Social Data.
        
        OUTPUT:
        Produce a Markdown verification report:
        - Candidate Name
        - Profile Found: Yes/No
        - DISCREPANCIES FOUND (List them clearly)
        - CONSISTENCY SCORE (0-100%)
        """

    def execute_tool_call(self, tool_call):
        fname = tool_call.function.name
        args = json.loads(tool_call.function.arguments)
        
        print(f"  [AGENT ACTION] Calling {fname} with {args}")
        
        if fname == "search_facebook_users":
            return mcp_server.search_facebook_users(**args)
        elif fname == "get_facebook_profile":
            return mcp_server.get_facebook_profile(**args)
        elif fname == "search_linkedin_people":
            return mcp_server.search_linkedin_people(**args)
        elif fname == "get_linkedin_profile":
            return mcp_server.get_linkedin_profile(**args)
        else:
            return json.dumps({"error": "Unknown tool"})

    def verify_cv(self, filename, cv_content):
        print(f"\n{'='*60}")
        print(f"STARTING VERIFICATION: {filename}")
        print(f"{'='*60}")
        
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": f"Please verify this CV content:\n\n{cv_content}"}
        ]
        
        for turn in range(5):
            response_msg = self.client.chat_completion(messages, self.tools)
            
            if not response_msg:
                print("[SYSTEM] Agent failed to respond.")
                break
            
            # --- CRITICAL FIX: Manually construct dict to avoid Pydantic errors ---
            msg_dict = {
                "role": "assistant",
                "content": response_msg.content
            }
            if response_msg.tool_calls:
                msg_dict["tool_calls"] = []
                for tc in response_msg.tool_calls:
                    msg_dict["tool_calls"].append({
                        "id": tc.id,
                        "type": tc.type,
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments
                        }
                    })
            
            messages.append(msg_dict)
            # ----------------------------------------------------------------------

            if response_msg.tool_calls:
                for tool_call in response_msg.tool_calls:
                    tool_result = self.execute_tool_call(tool_call)
                    
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": str(tool_result)
                    })
            else:
                print("\n[VERIFICATION REPORT]")
                print(response_msg.content)
                return response_msg.content
                
        return "Verification Incomplete."

# %% [markdown]
# ## 7. Main Execution

# %%
def main():
    agent = CVVerificationAgent(llm_client, tools_schema)
    
    results = {}
    for filename, content in CV_DATA.items():
        report = agent.verify_cv(filename, content)
        results[filename] = report
        time.sleep(2)

if __name__ == "__main__":
    main()

[SYSTEM] Configuration loaded.

STARTING VERIFICATION: CV_1.pdf
  [AGENT ACTION] Calling search_linkedin_people with {'location': 'Singapore', 'q': 'John Smith'}
  [AGENT ACTION] Calling get_linkedin_profile with {'person_id': 201}
  [AGENT ACTION] Calling search_facebook_users with {'q': 'John Smith'}

[VERIFICATION REPORT]
# Verification Report: John Smith

**Candidate Name:** John Smith
**Profile Found:** Yes (LinkedIn verified)

### 🚨 DISCREPANCIES FOUND

1.  **Role Inflation (ByteDance)**
    *   **CV Claim:** Engineer
    *   **Social Data:** Intern
    *   **Analysis:** The candidate claims a technical engineering role, whereas public records indicate an internship position.

2.  **Education Timeline Conflict**
    *   **CV Claim:** "2020 Present" AND "Graduated 2009"
    *   **Social Data:** 2005 – 2009
    *   **Analysis:** The CV contains contradictory dates. The "2020 Present" claim is false based on the verified graduation year of 2009.

3.  **Location Inaccuracy**
    *   

In [1]:
import requests
import time
import json

# ==========================================
# 1. CONFIGURATION
# ==========================================
API_KEY = "moltbook_sk_-MJliU38psOmkLtFb2swkfcsJZNfxPdw"
BASE_URL = "https://www.moltbook.com/api/v1"
TARGET_POST_ID = "47ff50f3-8255-4dee-87f4-2c3637c7351c"
NICKNAME = "nickname_12345"  # Ensure this matches your encoded student ID

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

class MoltbookSocialAgent:
    """
    An autonomous agent designed to participate in the Moltbook community
    by subscribing to submolts, upvoting content, and posting comments.
    """
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update(HEADERS)

    def log_action(self, step, message):
        """Standardized logging for agent actions."""
        timestamp = time.strftime('%H:%M:%S')
        print(f"[{timestamp}] [Step {step}] {message}")

    def run_workflow(self):
        print(f"--- Starting Agent: {NICKNAME} ---")

        # --- Task 1: Subscribe to /m/ftec5660 ---
        submolt = "ftec5660"
        self.log_action(1, f"Attempting to subscribe to: /m/{submolt}...")
        url_sub = f"{BASE_URL}/submolts/{submolt}/subscribe"
        res_sub = self.session.post(url_sub)
        
        if res_sub.status_code in [200, 201]:
            self.log_action(1, " Subscription successful!")
        else:
            self.log_action(1, f" Subscription failed: {res_sub.text}")

        time.sleep(1.5)  # Simulate reasoning delay/Avoid rate-limiting

        # --- Task 2: Upvote Post ---
        self.log_action(2, f"Attempting to upvote post: {TARGET_POST_ID}...")
        url_upvote = f"{BASE_URL}/posts/{TARGET_POST_ID}/upvote"
        res_upvote = self.session.post(url_upvote)
        
        if res_upvote.status_code in [200, 201]:
            self.log_action(2, " Upvote successful!")
        else:
            self.log_action(2, f" Upvote failed: {res_upvote.text}")

        time.sleep(1.5)

        # --- Task 3: Post Comment ---
        self.log_action(3, "Attempting to post task-specific comment...")
        url_comment = f"{BASE_URL}/posts/{TARGET_POST_ID}/comments"
        payload = {
            "content": f"Hello! This is agent {NICKNAME}. I have successfully completed FTEC5660 Homework 02 Part 2. #AgenticAI"
        }
        res_comment = self.session.post(url_comment, json=payload)
        
        if res_comment.status_code in [200, 201]:
            self.log_action(3, " Comment successful! All tasks complete.")
        else:
            self.log_action(3, f" Comment failed: {res_comment.text}")

        print(f"--- Agent Session Terminated ---")

# ==========================================
# 2. EXECUTION
# ==========================================
if __name__ == "__main__":
    agent = MoltbookSocialAgent()
    agent.run_workflow()

--- Starting Agent: nickname_12345 ---
[11:05:08] [Step 1] Attempting to subscribe to: /m/ftec5660...
[11:05:11] [Step 1]  Subscription successful!
[11:05:12] [Step 2] Attempting to upvote post: 47ff50f3-8255-4dee-87f4-2c3637c7351c...
[11:05:13] [Step 2]  Upvote successful!
[11:05:15] [Step 3] Attempting to post task-specific comment...
[11:05:16] [Step 3]  Comment successful! All tasks complete.
--- Agent Session Terminated ---
